# Sampling pose DKF trained on H3.6M

In [ ]:
%matplotlib inline
import os
import addpaths
from load import loadDataset
from common import insert_junk_entries
import os
import numpy as np
from scipy.signal import convolve2d

# Stupid hack to make parameter loading actually work
# (ugh, this is super brittle code)
import sys
del sys.argv[1:]
CONFIG_PATH = './chkpt-h36m/DKF_lr-8_0000e-04-vm-R-inf-structured-dh-50-ds-10-nl-relu-bs-20-ep-2000-rs-600-ttype-simple_gated-etype-mlp-previnp-False-ar-1_0000e+01-rv-5_0000e-02-nade-False-nt-5000-uid-config.pkl'
WEIGHT_PATH = './chkpt-h36m/DKF_lr-8_0000e-04-vm-R-inf-structured-dh-50-ds-10-nl-relu-bs-20-ep-2000-rs-600-ttype-simple_gated-etype-mlp-previnp-False-ar-1_0000e+01-rv-5_0000e-02-nade-False-nt-5000-uid-EP375-params.npz'
sys.argv.extend(['-reload', WEIGHT_PATH, '-params', CONFIG_PATH])
sys.argv.extend('-vm R -infm structured -ds 10 -dh 50'.split())

from parse_args_dkf import params
from utils.misc import removeIfExists,createIfAbsent,mapPrint,saveHDF5,displayTime
from stinfmodel_fast.dkf import DKF
import stinfmodel_fast.learning as DKF_learn
import stinfmodel_fast.evaluate as DKF_evaluate
from stinfmodel_fast.evaluate import sample

In [ ]:
dataset = loadDataset()

In [ ]:
params['savedir']+='-h36m'
# createIfAbsent(params['savedir'])

# Add dataset and NADE parameters to "params" which will become part of the
# model
for k in ['dim_observations','data_type']:
    params[k] = dataset[k]
mapPrint('Options: ',params)
if params['use_nade']:
    params['data_type']='real_nade'

# Remove from params
removeIfExists('./NOSUCHFILE')
reloadFile = params.pop('reloadFile')
pfile=params.pop('paramFile')
# paramFile is set inside the BaseClass in theanomodels
# to point to the pickle file containing params"""
assert os.path.exists(pfile),pfile+' not found. Need paramfile'
print 'Reloading trained model from : ',reloadFile
print 'Assuming ',pfile,' corresponds to model'
dkf  = DKF(params, paramFile = pfile, reloadFile = reloadFile)

In [ ]:
oodles_of_samples = sample(dkf, nsamples=50, T=1024)
sample_X, sample_Z = oodles_of_samples

In [ ]:
print('Output shape: %s' % str(sample_X.shape))
mu = dataset['h36m_mean'].reshape((1, 1, -1))
sigma = dataset['h36m_std'].reshape((1, 1, -1))
real_X = insert_junk_entries(sample_X * sigma + mu)
dest_dir = './generated/'
try:
    os.makedirs(dest_dir)
except OSError:
    pass
for i, sampled_times in enumerate(real_X):
    dest_fn = os.path.join(dest_dir, 'seq-%i.txt' % i)
    print('Saving %s' % dest_fn)
    np.savetxt(dest_fn, sampled_times, delimiter=',', fmt='%f')

In [ ]:
# Do the same thing, but smoothed
smooth_dest_dir = './generated-smooth/'
try:
    os.makedirs(smooth_dest_dir)
except OSError:
    pass
kernel = [0.1, 0.25, 0.3, 0.25, 0.1]
full_kernel = np.array(kernel).reshape((-1, 1))
for i, sampled_times in enumerate(real_X):
    dest_fn = os.path.join(smooth_dest_dir, 'seq-%i.txt' % i)
    print('Saving %s' % dest_fn)
    smooth_times = convolve2d(sampled_times, full_kernel, mode='valid')
    assert smooth_times.ndim == 2
    assert smooth_times.shape[1] == sampled_times.shape[1]
    np.savetxt(dest_fn, smooth_times, delimiter=',', fmt='%f')